# Задание 1.2

## Курузов Илья, 676

Выгрузим данные.

In [3]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y = True)

In [19]:
import numpy as np
classes = np.unique(y)
classes

array([0, 1, 2])

Разделим наши данные по классам.

In [29]:
arrays = []
for j in classes:
    arrays.append(np.array([i 
                            for ind, i in enumerate(X)
                                if y[ind] == j]))

In [30]:
arrays_mean = [i.mean(axis = 0) for i in arrays]
res = {}
for c1 in classes:
    for c2 in classes:
        if c1!= c2:
            name = str(c1) + "-" + str(c2)
            res[name] = arrays_mean[c1]-arrays_mean[c2]

### Различия между классами в среднем

Ниже приведена таблица, индексы по столбцам которой соответсвуют номерам признаков в исходной выборке, а названия колонок показывают между какими классами вычислялась разница. Каждый элемент этой таблицы есть разница между математическими ожиданиями признака для двух разных классов

In [31]:
import pandas as pd
pd.DataFrame(res)

,0-1,0-2,1-0,1-2,2-0,2-1
0,-0.930,-1.582,0.930,-0.652,1.582,0.652
1,0.658,0.454,-0.658,-0.204,-0.454,0.204
2,-2.798,-4.090,2.798,-1.292,4.090,1.292
3,-1.080,-1.780,1.080,-0.700,1.780,0.700


### Размер эффекта

In [32]:
from scipy import stats

#### Коэффициент Пирсона

In [35]:
k = 0
for x in X.T:
    print(k, stats.pearsonr(x, y)[0])
    k += 1

0 0.7825612318100814
1 -0.4266575607811244
2 0.9490346990083884
3 0.9565473328764029


Как можно заметить, метки класса имеют ярко выраженную линейную зависимость со 2-ым и 3-им признаками ($r\approx 0.95$). Также есть некоторая линейная зависимость с 0-ым признаком ($r = 0.78$). Наименьший по модулю коэффициент Пирсона наблюдается для 1-ого признака ($r = -0.43$).